# 文本和字节序列

## 介绍：
- 字符码位和字节表述
- bytes、bytearray和memoryview
- 编解码器
- 避免和处理编解码错误
- 处理文本文件
- 默认编码的陷阱和标准ＩＯ问题
- 规范化unicode文本，进行安全比较
- locale和pyUCA

## python3中的str对象获得的元素是unicode字符

## 码位和字节序列之间转换即编解码

In [41]:
s='café'
print(len(s))
b=s.encode('utf8')
print(b)
print(len(b))
b.decode('utf8')
print(b)

4
b'caf\xc3\xa9'
5
b'caf\xc3\xa9'


## bytes或bytearray对象是介于０－２５５之间的整数，切片类型与原类型相同

In [42]:
cafe=bytes('café',encoding='utf-8')
print(cafe)
print(cafe[0])
print(cafe[:1])
cafe_arr=bytearray(cafe)
print(cafe_arr)
print(cafe_arr[-1:])

b'caf\xc3\xa9'
99
b'c'
bytearray(b'caf\xc3\xa9')
bytearray(b'\xa9')


## bytes,fromhex可用于解析十六进制数字

In [6]:
print(bytes.fromhex('31 4B CE A9'))

b'1K\xce\xa9'


## 构建bytes、bytearray:
- 使用str和encoding关键字作为参数
- 一个可迭代对象，提供０～２５５之间的数值
- 一个实现了缓冲协议的对象(bytes,bytearray,memoryview,array.array)

In [7]:
import array
numbers=array.array('h',[-2,-1,0,1,2])
octets=bytes(numbers)
print(octets)

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'


## struct模块把打包的字节序列转换成不同类型字段组成的元组，能处理bytes,bytearray,memoryview对象

In [26]:
import struct

fmt='<4s1s4sH'
header=bytes('justatest00',encoding='utf-8')
print(struct.unpack(fmt,header))

(b'just', b'a', b'test', 12336)


## 尝试不同的编解码器

In [28]:
for codec in ['latin_1','utf-8','utf-16']:
    print(codec,'EL Nino'.encode(codec),sep='\t')

latin_1	b'EL Nino'
utf-8	b'EL Nino'
utf-16	b'\xff\xfeE\x00L\x00 \x00N\x00i\x00n\x00o\x00'


## 处理UnicodeEncodeError

In [33]:
city='São Paulo'
# city.encode('cp437')
city.encode('cp437',errors='ignore')
city.encode('cp437',errors='replace')
city.encode('cp437',errors='xmlcharrefreplace')

b'S&#227;o Paulo'

## 处理UnicodeDecodeError

In [40]:
octets=b'Monrt\xe9al'
octets.decode('cp1252')
octets.decode('iso8859_7')
octets.decode('koi8_r')
octets.decode('utf-8',errors='replace')

'Monrt�al'

## 处理SyntaxError
- 打开py文件出错时在文件顶部加入encoding注释
- \#encoding:utf-8

## 找出字节序列的源码：使用chardet!

## 处理文本文件
- 尽早将输入的字节序列解码成字符串
- 在业务逻辑中只处理字符串对象
- 尽量晚把字符串编码成字节序列
- 需要在在不同平台运行代码，则打开文件时应始终明确encoding参数

## 使用unicodedata.normalize函数进行Unicode字符串规范化

In [45]:
from unicodedata import normalize
s1='café'
s2='cafe\u0301'
print(len(s1),len(s2))
print(len(normalize('NFC',s1)),len(normalize('NFC',s2)))
print(len(normalize('NFD',s1)),len(normalize('NFD',s2)))

4 5
4 4
5 5


## 大小写折叠

In [46]:
caps='bjBLBJLHJaiUBsbdjbB'
micro=caps.casefold()
print(micro)

bjblbjlhjaiubsbdjbb


## 规范化文本匹配

In [51]:
def nfc_equal(str1,str2):
    return normalize('NFC',str1)==normalize('NFC',str2)
def fold_equal(str1,str2):
    return normalize('NFC',str1).casefold()==normalize('NFC',str2).casefold()

s1='cafe'
s2='Cafe'

print(nfc_equal(s1,s2))
print(fold_equal(s1,s2))

False
True


## Unicode文本排序
使用pyuca.Collator.sort_key方法进行排序，保证对不同语言按照当地规则进行排序

In [55]:
import pyuca
coll=pyuca.Collator()
sort_list_cn=['张三','李四','王五','王三']
sorted_list_cn=sorted(sort_list_cn,key=coll.sort_key)
print(sorted_list_cn)

['张三', '李四', '王三', '王五']


## 中文排序貌似还是locale.strxfrm作为排序键更靠谱

In [60]:
import locale
locale.setlocale(locale.LC_COLLATE,'zh_CN.UTF-8')
sort_list_cn=['张三','李四','王五','王三']
sorted_list_cn=sorted(sort_list_cn,key=locale.strxfrm)
print(sort_list_cn)

['张三', '李四', '王五', '王三']
